In [2]:
import numpy as np
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup as bs
#from google.colab import files
from tqdm import *
#courses = pd.read_csv('courses_all.csv')
#career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])

In [2]:
eng_list = []

In [3]:
for i in tqdm(range(19)):
    page = rq.get('https://hospitality.careers360.com/colleges/list-of-hospitality-tourism-colleges-in-India?page='+str(i))
    soup = bs(page.content,'html.parser')
    vals = soup.find_all('div',class_='title')
    for val in vals:
        try:
            eng_list.append(val.a['href'])
        except:
            pass

100%|██████████| 19/19 [01:25<00:00,  4.50s/it]


In [4]:
pd.DataFrame(eng_list).to_csv('hotel_list.csv')

In [4]:
courses = pd.read_csv('hotel_courses.csv')

In [ ]:
idx = 0
for link in tqdm(eng_list):
    try:
        page = rq.get(link)
        soup = bs(page.content,'html.parser')
        courses.loc[idx]=[link,soup.find_all('h4',class_='blockSubHeading')[0].get_text()[-2:],'https://hospitality.careers360.com/'+soup.find_all('ul',class_='list-tabs')[0].find_all('li')[1].a['href']]
        idx+=1
    except:
        print(f'error in {idx} and {link}')
        pass

 45%|████▍     | 246/552 [11:31<14:19,  2.81s/it]

In [ ]:
courses.to_csv('hotel_courses.csv')

In [9]:
from threading import Thread
def scrapper(start, end):
    career360_courses = pd.DataFrame(columns=['College','City','State','Course','Duration','Study mode','Approved intake','Course level','Course Fee','Approval','Eligiblity','Course_Detail'])
    indx = 0
    cnt = start - 1
    for header_course in tqdm(courses['2'][start:end]):
        cnt += 1
        try:
            num = int(np.ceil(int(courses['1'][cnt])/10))
        except:
            continue
    #     print(f'num = {num}')
        for page_num in tqdm(range(num)):
            link = header_course + '?levelId=all&courseNid=all-all&page='+str(page_num)
    #         print(link)
            page = rq.get(link)
            course_soup = bs(page.content, 'html.parser')
        for course_link in course_soup.find_all('span',class_='readmore'):
            course_link = 'https://hospitality.careers360.com/' + course_link.a['href']
            #             print(f'course_link = {course_link}')
            page1 = rq.get(course_link)
            soup = bs(page1.content,'html.parser')
            career360_courses.loc[indx]=None

            try:
                career360_courses.loc[indx]['Course'] = soup.find_all('h2',class_='block-title')[0].get_text()
                career360_courses.loc[indx]['College']=soup.find_all('h1',class_='titleNameCol')[0].get_text()
                career360_courses.loc[indx]['City'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[0].get_text()
                career360_courses.loc[indx]['State'] = soup.find_all('ul',class_='clg-info')[0].find_all('li')[0].find_all('a')[1].get_text()
            except:
                pass

            try:
                career360_courses.loc[indx]['Eligiblity'] = soup.find_all('span',class_='more-eligibility')[0].get_text()[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Eligiblity'] = soup.find_all('div',class_='default-elig')[0].get_text()
                except:
                    career360_courses.loc[indx]['Eligiblity'] = None

            try:
                career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('span',class_='moreCourseDetails')[0].p.get_text())[:-11]
            except:
                try:
                    career360_courses.loc[indx]['Course_Detail'] = (soup.find_all('div',class_='default-course-details')[0].get_text())
                except:
                    career360_courses.loc[indx]['Course_Detail'] = None

            for row in soup.find_all('div',class_='coursesPageLableInnerSec'):
                career360_courses.loc[indx][row.strong.get_text()] = row.p.get_text()
            indx+=1
    career360_courses.to_csv(f'career360_scrap_dm_hotel({start}-{end}).csv')

In [10]:
siz = courses.shape[0]
numer = int(siz/20)
prev = 0
def threader():
    threads = []
    for i in range(20):
        if i!=19:
            thread = Thread(target=scrapper, args = (i*numer,(i+1)*numer))
            threads.append(thread)
            thread.start()
        else:
            thread = Thread(target=scrapper, args = (19*numer,siz))
            threads.append(thread)
            thread.start()        
    
    for thread in threads:  # iterates over the threads
        thread.join()
                    
    print('done')

In [11]:
threader()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/27 [00:00<?, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]



  0%|          | 0/27 [00:00<?, ?it/s]




  0%|          | 0/1 [00:00<?, ?it/s]





  0%|          | 0/27 [00:00<?, ?it/s]






  0%|          | 0/1 [00:00<?, ?it/s]







  0%|          | 0/27 [00:00<?, ?it/s]








  0%|          | 0/1 [00:00<?, ?it/s]









  0%|          | 0/27 [00:00<?, ?it/s]










  0%|          | 0/1 [00:00<?, ?it/s]











  0%|          | 0/27 [00:00<?, ?it/s]












  0%|          | 0/4 [00:00<?, ?it/s]













  0%|          | 0/27 [00:00<?, ?it/s]














  0%|          | 0/1 [00:00<?, ?it/s]















  0%|          | 0/27 [00:00<?, ?it/s]
















  0%|          | 0/1 [00:00<?, ?it/s]

















  0%|          | 0/27 [00:00<?, ?it/s]


















  0%|          | 0/1 [00:00<?, ?it/s]



















  0%|          | 0/27 [00:00<?, ?it/s]




















  0%|          | 0/2

  0%|          | 0/1 [00:00<?, ?it/s]













100%|██████████| 2/2 [00:05<00:00,  2.60s/it]































  3%|▎         | 1/38 [00:05<03:09,  5.13s/it]






















  0%|          | 0/1 [00:00<?, ?it/s]














  4%|▎         | 1/27 [00:05<02:15,  5.22s/it]























  0%|          | 0/1 [00:00<?, ?it/s]
















  4%|▎         | 1/27 [00:05<02:20,  5.40s/it]
























  0%|          | 0/1 [00:00<?, ?it/s]



  4%|▎         | 1/27 [00:05<02:29,  5.75s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]


 11%|█         | 3/27 [00:05<00:46,  1.94s/it]


 15%|█▍        | 4/27 [00:05<00:33,  1.46s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]












  4%|▎         | 1/27 [00:05<02:25,  5.60s/it]



























  0%|          | 0/1 [00:00<?, ?it/s]

  4%|▎         | 1/27 [00:05<02:32,  5.88s/it]




























  4%|▎         | 1/27 [00:05<02:34,  5.93s/

  0%|          | 0/1 [00:00<?, ?it/s]









 11%|█         | 3/27 [00:13<01:47,  4.49s/it]























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.99s/it]

































  7%|▋         | 2/27 [00:13<02:48,  6.72s/it]























  0%|          | 0/1 [00:00<?, ?it/s]










  4%|▎         | 1/27 [00:13<05:52, 13.55s/it]
























  0%|          | 0/2 [00:00<?, ?it/s]














 11%|█         | 3/27 [00:13<01:51,  4.66s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.47s/it]



















  7%|▋         | 2/27 [00:15<03:12,  7.70s/it][A

























  0%|          | 0/1 [00:00<?, ?it/s]


















  7%|▋         | 2/27 [00:15<03:16,  7.86s/it]


























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.58s/it]








100%|██████████| 1/1 [00:03<00:00,  3.35s/it]





















 11%|█         | 3/27 [00:22<02:56,  7.37s/it][A



























  0%|          | 0/2 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:03<00:00,  3.62s/it]






































 11%|█         | 3/27 [00:21<02:53,  7.24s/it]



























  0%|          | 0/1 [00:00<?, ?it/s]





 22%|██▏       | 6/27 [00:22<01:17,  3.68s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]




















 14%|█▍        | 1/7 [00:02<00:17,  2.87s/it]
























100%|██████████| 1/1 [00:02<00:00,  2.21s/it]







































 15%|█▍        | 4/27 [00:22<02:10,  5.67s/it]














 19%|█▊        | 5/27 [00:22<01:40,  4.55s/it]




























  0%|          | 0/1 [00:00<?, ?it/s]























100%|██████████| 1/1 [00:02<00:00,  2.52s/it]













































100%|██████████| 1/1 [0

























100%|██████████| 1/1 [00:02<00:00,  2.45s/it]















































100%|██████████| 1/1 [00:02<00:00,  2.47s/it]


























 30%|██▉       | 8/27 [00:29<01:10,  3.70s/it]






























  0%|          | 0/2 [00:00<?, ?it/s]


























100%|██████████| 1/1 [00:02<00:00,  2.61s/it]


















































100%|██████████| 1/1 [00:03<00:00,  3.02s/it]













































100%|██████████| 1/1 [00:03<00:00,  3.60s/it]









































 50%|█████     | 4/8 [00:11<00:11,  2.98s/it]




















 43%|████▎     | 3/7 [00:10<00:14,  3.61s/it]























100%|██████████| 1/1 [00:03<00:00,  3.22s/it]













































100%|██████████| 1/1 [00:03<00:00,  3.90s/it]












































100%|██████████| 1/1 [00:03<00:00,  3.47s/it]




































...

 74%|███████▍  | 20/27 [03:31<01:13, 10.56s/it]

























  0%|          | 0/1 [00:00<?, ?it/s]



















100%|██████████| 1/1 [00:02<00:00,  2.44s/it]







































100%|██████████| 1/1 [00:02<00:00,  2.80s/it]






































 50%|█████     | 2/4 [00:06<00:06,  3.26s/it]











 89%|████████▉ | 24/27 [03:32<00:26,  8.85s/it]
























  0%|          | 0/6 [00:00<?, ?it/s]





















100%|██████████| 1/1 [00:02<00:00,  2.18s/it]









































100%|██████████| 1/1 [00:03<00:00,  3.46s/it]







































100%|██████████| 1/1 [00:02<00:00,  2.65s/it]







































100%|██████████| 1/1 [00:02<00:00,  2.31s/it]





































 86%|████████▌ | 6/7 [00:15<00:02,  2.55s/it]








 78%|███████▊  | 21/27 [03:34<01:01, 10.20s/it]





















  0%|          | 0/1 [00:00<?, ?it/s]



















 89%|████████▉ | 24/27 [03:43<00:27,  9.29s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]














 67%|██████▋   | 4/6 [00:11<00:05,  2.78s/it]

 81%|████████▏ | 22/27 [03:43<00:50, 10.18s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]















100%|██████████| 1/1 [00:03<00:00,  3.73s/it]
































100%|██████████| 1/1 [00:03<00:00,  3.27s/it]




























 82%|████████▏ | 31/38 [03:44<00:50,  7.23s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]















100%|██████████| 2/2 [00:04<00:00,  2.16s/it]






















 85%|████████▌ | 23/27 [03:44<00:39,  9.76s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]















 25%|██▌       | 1/4 [00:03<00:09,  3.28s/it]





 93%|█████████▎| 25/27 [03:45<00:18,  9.01s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]








 78%|███████▊  | 21/27 [03:45<01:04, 10.73s/it]





















  0%|         

 93%|█████████▎| 25/27 [03:52<00:18,  9.30s/it]


















  0%|          | 0/2 [00:00<?, ?it/s]







 93%|█████████▎| 25/27 [03:52<00:18,  9.32s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]















 10%|█         | 1/10 [00:02<00:21,  2.43s/it]
















100%|██████████| 1/1 [00:02<00:00,  2.45s/it]

































 17%|█▋        | 1/6 [00:02<00:11,  2.33s/it]








 81%|████████▏ | 22/27 [03:53<00:52, 10.60s/it]



















 85%|████████▌ | 23/27 [03:53<00:40, 10.15s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]














100%|██████████| 4/4 [00:11<00:00,  2.86s/it]

















100%|██████████| 27/27 [03:53<00:00,  8.65s/it]




 89%|████████▉ | 24/27 [03:54<00:29,  9.77s/it]



















  0%|          | 0/1 [00:00<?, ?it/s]











 85%|████████▌ | 23/27 [03:54<00:40, 10.20s/it]




















  0%|          | 0/1 [00:00<?, ?it/s]





 93%|█████████▎| 25/27 [03:54<00:18,  9.40s/

100%|██████████| 1/1 [00:02<00:00,  2.25s/it]


























100%|██████████| 6/6 [00:15<00:00,  2.51s/it]





















100%|██████████| 27/27 [04:05<00:00,  9.10s/it]





















100%|██████████| 1/1 [00:03<00:00,  3.04s/it]























100%|██████████| 1/1 [00:02<00:00,  2.43s/it]











 85%|████████▌ | 23/27 [04:06<00:42, 10.72s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 2/2 [00:04<00:00,  2.38s/it]













 89%|████████▉ | 24/27 [04:06<00:30, 10.29s/it]












  0%|          | 0/1 [00:00<?, ?it/s]










100%|██████████| 1/1 [00:02<00:00,  2.66s/it]


















 92%|█████████▏| 35/38 [04:06<00:21,  7.05s/it]












  0%|          | 0/1 [00:00<?, ?it/s]









 60%|██████    | 6/10 [00:16<00:11,  2.80s/it]






100%|██████████| 27/27 [04:07<00:00,  9.16s/it]







 96%|█████████▋| 26/27 [04:08<00:09,  9.54s/it]












  0%|          | 0/1 [00:00<?, ?it/s]

100%|████████

done
